In [1]:
import tensorflow
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from collections import deque

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return lh, rh

In [6]:
def draw_styled_landmarks(image, results):

    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('AlphabetData/')
# # Actions that we try to detect
actions = np.array(os.listdir(DATA_PATH))
print(actions)
# # Thirty videos worth of data
no_sequences = 360
# # Videos are going to be 30 frames in length
sequence_length = 30 
# # Folder start
start_folder = 1

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [8]:
new_model = tensorflow.keras.models.load_model('AllAlphabets.h5')

In [17]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.98
cap = cv2.VideoCapture("C:/Users/Hassan Ahmed/GP/AlphabetVideos/L.mp4")
cv2.namedWindow('Webcam', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Webcam', 720, 1280)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
       # flipped = cv2.flip(frame,1)
                        
        # Make detections
    
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        
        lh, rh = extract_keypoints(results)
        keypoints = np.concatenate([lh, rh])
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        if  np.sum(lh) == 0 and np.sum(rh) == 0 : 
            sentence = []
            sequence = []
            predictions = []
        else:
            if len(sequence) == sequence_length:
                res = new_model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                if res[np.argmax(res)] > threshold: 
                    sentence.append(actions[np.argmax(res)])
            
                    

            sentence = sentence[-1:]
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)




        
        # Show to screen
        cv2.imshow('Webcam', image)


        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows() 

1/1 [==============================] - 0s 30ms/step
L
1/1 [==============================] - 0s 31ms/step
L
1/1 [==============================] - 0s 29ms/step
L
1/1 [==============================] - 0s 29ms/step
L
1/1 [==============================] - 0s 30ms/step
L
1/1 [==============================] - 0s 30ms/step
L
1/1 [==============================] - 0s 30ms/step
L
1/1 [==============================] - 0s 29ms/step
L
1/1 [==============================] - 0s 30ms/step
L
1/1 [==============================] - 0s 30ms/step
L


In [10]:
    cap.release()
    cv2.destroyAllWindows() 